# 调用自定义类

[![下载Notebook](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_notebook.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/experts/zh_cn/debug/mindspore_ms_class.ipynb)&emsp;[![下载样例代码](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_download_code.png)](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/notebook/master/tutorials/experts/zh_cn/debug/mindspore_ms_class.py)&emsp;[![查看源文件](https://mindspore-website.obs.cn-north-4.myhuaweicloud.com/website-images/master/resource/_static/logo_source.png)](https://gitee.com/mindspore/docs/blob/master/tutorials/experts/source_zh_cn/debug/ms_class.ipynb)

## 概述

在静态图模式下，通过使用ms_class修饰自定义类，用户可以创建、调用该自定义类的实例，并且可以获取其属性和方法。

ms_class应用于静态图模式，扩充完善静态图编译语法的支持范围。在动态图模式即PyNative模式下，ms_class的使用不影响PyNative模式的执行逻辑。

本文档主要介绍ms_class的使用方法和使用须知，以便您可以更有效地使用ms_class功能。

## ms_class修饰自定义类

自定义类使用了@ms_class后，支持创建、调用该自定义类的实例，获取其属性和方法。

ms_class仅支持修饰自定义类，不支持nn.Cell和非class类型。需要注意的是，类继承时，如果父类使用了ms_class，子类也会具有ms_class的能力。

In [1]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor, ms_class

@ms_class
class InnerNet:
    value = Tensor(np.array([1, 2, 3]))

class Net(nn.Cell):
    def construct(self):
        return InnerNet.value

context.set_context(mode=context.GRAPH_MODE)
net = Net()
out = net()
print(out)

[1 2 3]


ms_class支持自定义类嵌套使用、自定义类与nn.Cell嵌套使用的场景。

In [2]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor, ms_class

@ms_class
class Inner:
    def __init__(self):
        self.value = Tensor(np.array([1, 2, 3]))

@ms_class
class InnerNet:
    def __init__(self):
        self.inner = Inner()

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.inner_net = InnerNet()

    def construct(self):
        out = self.inner_net.inner.value
        return out

context.set_context(mode=context.GRAPH_MODE)
net = Net()
out = net()
print(out)

[1 2 3]


## 获取自定义类的属性和方法

支持通过类名调用类的属性，不支持通过类名调用类的方法；对于类的实例，支持调用其属性和方法。

不支持调用私有属性和魔术方法，所调用的方法函数须在静态图编译支持的语法范围内。

In [3]:
import mindspore.nn as nn
import mindspore.common.dtype as mstype
from mindspore import context, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, val):
        self.number = val

    def act(self, x, y):
        return self.number * (x + y)

class Net(nn.Cell):
    def __init__(self):
        super(Net, self).__init__()
        self.inner_net = InnerNet(2)

    def construct(self, x, y):
        return self.inner_net.number + self.inner_net.act(x, y)

x = Tensor(2, dtype=mstype.int32)
y = Tensor(3, dtype=mstype.int32)
net = Net()
out = net(x, y)
print(out)

12


## 创建自定义类的实例

创建自定义类的实例时，参数要求为常量。

In [4]:
import numpy as np
import mindspore.nn as nn
from mindspore import context, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, val):
        self.number = val + 3

class Net(nn.Cell):
    def construct(self):
        net = InnerNet(2)
        return net.number

net = Net()
out = net()
print(out)

5


## 调用自定义类的实例

调用自定义类的实例时，将会调用该类的`__call__`函数方法。如果该类没有定义`__call__`函数，将会报错提示。

In [5]:
import numpy as np
import mindspore.nn as nn
import mindspore.common.dtype as mstype
from mindspore import context, Tensor, ms_class

@ms_class
class InnerNet:
    def __init__(self, number):
        self.number = number

    def __call__(self, x, y):
        return self.number * (x + y)

class Net(nn.Cell):
    def construct(self, x, y):
        net = InnerNet(2)
        out = net(x, y)
        return out

x = Tensor(2, dtype=mstype.int32)
y = Tensor(3, dtype=mstype.int32)
net = Net()
out = net(x, y)
print(out)

10
